<a href="https://colab.research.google.com/github/Pengyu-gis/Historical-Aerial-Photos/blob/main/Object_Detecion/YoloV11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Convert xml to txt

In [ ]:
import xml.etree.ElementTree as ET
import os
from PIL import Image

# Setting the folder path
xml_folder = 'Dataset/labels'  # XML Dataset
txt_folder = 'Dataset/labels-txt'  # YOLO Label Dataset
class_names = ['1']  # assume class is '1'

# Create the output folder, if it doesn't exist
os.makedirs(txt_folder, exist_ok=True)

def convert_xml_to_yolo(xml_file, txt_file):
    # Parsing XML files
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Get the width and height of the image
    size = root.find('size')
    img_width = int(size.find('width').text)
    img_height = int(size.find('height').text)

    # Open txt file ready to write
    with open(txt_file, 'w') as f:
        for obj in root.iter('object'):
            # Get Class ID
            class_name = obj.find('name').text
            # class_id = class_names.index(class_name)  # assue class is in the class_names list

            # Get bounding box information
            bndbox = obj.find('bndbox')
            xmin = float(bndbox.find('xmin').text)
            ymin = float(bndbox.find('ymin').text)
            xmax = float(bndbox.find('xmax').text)
            ymax = float(bndbox.find('ymax').text)

            # Calculate coordinates in YOLO format
            x_center = (xmin + xmax) / 2.0 / img_width
            y_center = (ymin + ymax) / 2.0 / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height

            # Write tags to txt file
            f.write(f"{class_name} {x_center} {y_center} {width} {height}\n")

# Processing each XML file
for xml_file in os.listdir(xml_folder):
    if xml_file.endswith('.xml'):
        # Get XML File path
        xml_path = os.path.join(xml_folder, xml_file)

        # Get the image filename (assuming the image is in .tif format)
        img_name = xml_file.replace('.xml', '.tif')  # Assume the image is in .tif format
        # Generate the corresponding label file in YOLO format
        txt_path = os.path.join(txt_folder, img_name.replace('.tif', '.txt'))
        convert_xml_to_yolo(xml_path, txt_path)
        print(f"Created{xml_path}")

print("All label files have been converted to YOLO format.")

KeyboardInterrupt: 

## Convert tif to jpg

In [ ]:
import os
from PIL import Image

# Set the folder paths
input_folder = 'Dataset/images'  # Path to the folder containing .tif files
output_folder = 'Dataset/img-jpg'  # Path to the output folder for .jpg files

# Create the output folder (if it doesn't exist)
os.makedirs(output_folder, exist_ok=True)

# Loop through all .tif files in the folder
for tif_file in os.listdir(input_folder):
    if tif_file.endswith('.tif'):
        # Get the full path of the .tif file
        tif_path = os.path.join(input_folder, tif_file)

        # Open the .tif file
        img = Image.open(tif_path)

        # Get the file name (without extension)
        file_name = os.path.splitext(tif_file)[0]

        # Define the output path for the .jpg file
        jpg_path = os.path.join(output_folder, f"{file_name}.jpg")

        # Convert and save as .jpg
        img.convert("RGB").save(jpg_path, "JPEG")

        print(f"Converted {tif_file} to {file_name}.jpg")

print("All .tif files have been converted to .jpg format.")

In [ ]:
import os

# Set the folder path
folder_path = 'Dataset/labels-txt'  # Replace with your folder path

# Get all files in the folder and filter for .txt files
jpg_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.lower().endswith('.txt')]

# Count the number of .txt files
num_jpg_files = len(jpg_files)

print(f"There are {num_jpg_files} .txt files in the folder.")

### Dividing the data set

In [ ]:
import os
import random
import shutil

# Set source folder paths
img_folder = 'Dataset/img-jpg'  # Replace with the path to your image folder
label_folder = 'Dataset/labels-txt'  # Replace with the path to your label folder (txt label files)

# Set destination folder paths
train_img_folder = 'Dataset_2_25/yolo/dataset/train/images'
train_label_folder = 'Dataset_2_25/yolo/dataset/train/labels'
val_img_folder = 'Dataset_2_25/yolo/dataset/val/images'
val_label_folder = 'Dataset_2_25/yolo/dataset/val/labels'

# Create destination folders
os.makedirs(train_img_folder, exist_ok=True)
os.makedirs(train_label_folder, exist_ok=True)
os.makedirs(val_img_folder, exist_ok=True)
os.makedirs(val_label_folder, exist_ok=True)

# Get all .jpg image files from the image folder
image_files = [f for f in os.listdir(img_folder) if f.endswith('.jpg')]

# Shuffle the image files
random.shuffle(image_files)

# Split the dataset into 80% training and 20% validation
train_size = int(0.8 * len(image_files))
train_images = image_files[:train_size]
val_images = image_files[train_size:]

# Copy images and corresponding labels to the new folders
for img_file in train_images:
    # Get paths of the image and its label
    img_path = os.path.join(img_folder, img_file)
    label_path = os.path.join(label_folder, img_file.replace('.jpg', '.txt'))  # Labels are in .txt format

    # Copy image file to the training images folder
    shutil.copy(img_path, os.path.join(train_img_folder, img_file))

    # Copy the corresponding label file to the training labels folder
    shutil.copy(label_path, os.path.join(train_label_folder, img_file.replace('.jpg', '.txt')))

for img_file in val_images:
    # Get paths of the image and its label
    img_path = os.path.join(img_folder, img_file)
    label_path = os.path.join(label_folder, img_file.replace('.jpg', '.txt'))  # Labels are in .txt format

    # Copy image file to the validation images folder
    shutil.copy(img_path, os.path.join(val_img_folder, img_file))

    # Copy the corresponding label file to the validation labels folder
    shutil.copy(label_path, os.path.join(val_label_folder, img_file.replace('.jpg', '.txt')))

print("Dataset split completed.")

**Downloading YOLO**

In [ ]:
# Install the ultralytics package from PyPI
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.9/921.9 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninsta

### Training

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.yaml")  # build a new model from YAML
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data="/content/drive/MyDrive/Dataset_2_25/yolo/dataset/data.yaml", epochs=100, imgsz=256)

# 保存模型
model.save('best.pt')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 155MB/s]


Transferred 499/499 items from pretrained weights
Ultralytics 8.3.80 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.yaml, data=/content/drive/MyDrive/Dataset_2_25/yolo/dataset/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=yolo11n.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sav

100%|██████████| 755k/755k [00:00<00:00, 42.2MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

...

    100/100     0.449G      1.243     0.7484      1.107         28        256: 100%|██████████| 113/113 [00:12<00:00,  9.26it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:03<00:00,  7.05it/s]

                   all        676       2182      0.911       0.78      0.853      0.581



100 epochs completed in 0.457 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.4MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.4MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.80 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:04<00:00,  5.33it/s]


                   all        676       2182       0.91      0.782      0.853      0.581
              building        676       2182       0.91      0.782      0.853      0.581
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/train


### Validation

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load an official model
model = YOLO("/content/best.pt")  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/drive/MyDrive/Dataset_2_25/yolo/dataset/val/labels.cache... 678 images, 0 backgrounds, 2 corrupt: 100%|██████████| 678/678 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/MyDrive/Dataset_2_25/yolo/dataset/val/images/000000002781.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 2. Possible class labels are 0-1
val: WARNING ⚠️ /content/drive/MyDrive/Dataset_2_25/yolo/dataset/val/images/000000002783.jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 2. Possible class labels are 0-1



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 43/43 [00:05<00:00,  7.95it/s]


                   all        676       2182      0.901      0.785      0.849      0.577
                class1        676       2182      0.901      0.785      0.849      0.577
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val


array([    0.57666,     0.57666])

**Prediction**

In [ ]:
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Load the model
model = YOLO("/content/best.pt")  # Replace with your model path

# Define image path and label path
image_path = "yolo/dataset/val/images/000000003287.jpg"
label_path = "yolo/dataset/val/labels/000000003287.txt"

# Read the image
image = Image.open(image_path)
image_np = np.array(image)

# Read the label file
with open(label_path, 'r') as file:
    labels = file.readlines()

# Parse the labels
for label in labels:
    parts = label.strip().split()
    class_id = int(parts[0])
    x_center, y_center, width, height = map(float, parts[1:])

    # Convert normalized coordinates to pixel coordinates
    img_width, img_height = image.size
    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height

    # Calculate top-left and bottom-right coordinates of the bounding box
    x1 = int(x_center - width / 2)
    y1 = int(y_center - height / 2)
    x2 = int(x_center + width / 2)
    y2 = int(y_center + height / 2)

    # Draw the ground-truth bounding box
    draw = ImageDraw.Draw(image)
    draw.rectangle([x1, y1, x2, y2], outline="red", width=2)

    # Draw the class label
    font = ImageFont.load_default()
    draw.text((x1, y1), str(class_id), fill="red", font=font)

# Run inference with the model
results = model(image_path)

# Get prediction results
for result in results:
    # Draw predicted bounding boxes
    boxes = result.boxes.xyxy  # Get predicted bounding box coordinates
    for box in boxes:
        x1, y1, x2, y2 = box
        draw.rectangle([x1, y1, x2, y2], outline="blue", width=2)
        draw.text((x1, y1), "Pred", fill="blue", font=font)

# Display the result
plt.imshow(image)
plt.axis('off')
plt.show()


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("/content/best.pt")  # trained YOLO11n model

# Run batched inference on a list of images
results = model(["yolo/dataset/train/images/000000002851.jpg"])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    # result.save(filename="result.jpg")  # save to disk